In [68]:
import loguru
import numpy as np
import pandas as pd
from logger import FhpLogger
from sklearn import datasets
from sklearn.dummy import DummyRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler

In [7]:
data = datasets.fetch_california_housing()

df = pd.DataFrame(data=data.data, columns=data.feature_names) 
df[data["target_names"][0] ] = data["target"]

In [8]:
df

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422
...,...,...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09,0.781
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21,0.771
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22,0.923
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32,0.847


In [54]:
# Model Evaluation w/ Cross Validation
def modelEval(train, test,feature="MedHouseVal" ,model_id = 'dummy'):
    
    # Input: Feature & Target DataFrame

    # Split feature/target variable
    y = train[feature].copy()
    X = train.copy()
    del X[feature]     # remove target variable
    
    y_test = test[feature].copy()
    X_test = test.copy()
    del X_test[feature]
    
    # Pick Model 
    if(model_id == 'dummy'):    model = DummyRegressor()
    if(model_id == 'rf'):    model = RandomForestRegressor(n_estimators=10,random_state=10)
    
    ''' Parameter Based Cross Validation (No Pipeline)'''
#     gscv = GridSearchCV(model,param_grid,cv=5)
#     gscv.fit(X,y)
#     results = pd.DataFrame(gscv.cv_results_)
#     scores = np.array(results.mean_test_score).reshape(7,7)
    
#     # plot the cross validation mean scores
#     heatmap1(scores,xlabel='lamda',xticklabels=param_grid['lamd'],
#                     ylabel='alpha',yticklabels=param_grid['alph'])
    
    ''' Standard Cross Validation '''
    cv_score = np.sqrt(-cross_val_score(model,X,y,cv=5,scoring='neg_mean_squared_error'))
    print(f"Scores: {cv_score}, Mean: {cv_score.mean()}, std: {cv_score.std()}" )

    preds = model.fit(X, y)
    preds = model.predict(X_test)
    test_score = mean_squared_error(y_test, preds)
    print(f"Test score: {test_score}")

# function that imputes a dataframe 
def impute_knn(df):
    
    # imputation with KNN unsupervised method

    # separate dataframe into numerical/categorical
    ldf = df.select_dtypes(include=[np.number])           # select numerical columns in df
    ldf_putaside = df.select_dtypes(exclude=[np.number])  # select categorical columns in df
    # define columns w/ and w/o missing data
    cols_nan = ldf.columns[ldf.isna().any()].tolist()         # columns w/ nan 
    cols_no_nan = ldf.columns.difference(cols_nan).values     # columns w/o nan 

    for col in cols_nan:                
        imp_test = ldf[ldf[col].isna()]   # indicies which have missing data will become our test set
        imp_train = ldf.dropna()          # all indicies which which have no missing data 
        model = KNeighborsRegressor(n_neighbors=5)  # KNR Unsupervised Approach
        knr = model.fit(imp_train[cols_no_nan], imp_train[col])
        ldf.loc[df[col].isna(), col] = knr.predict(imp_test[cols_no_nan])
    
    return pd.concat([ldf,ldf_putaside],axis=1)

In [69]:
data = datasets.fetch_california_housing()

df = pd.DataFrame(data=data.data, columns=data.feature_names) 
df[data["target_names"][0] ] = data["target"]

df2 = impute_knn(df)

trdata,tedata = train_test_split(df2,test_size=0.3,random_state=42)
trdata , tedata = trdata.copy(), tedata.copy()
del df2, df
# trdata_upd : training data w/ removed outliers
maxval2 = trdata[data["target_names"][0]].max() # get the maximum value
trdata_upd = trdata[trdata[data["target_names"][0] ] != maxval2].copy()
tedata_upd = tedata[tedata[data["target_names"][0] ] != maxval2].copy()
# trdata_upd.hist(bins=60, figsize=(15,9),color=color1);
# plt.show() # looks like its completely removed.

# Make a feature that contains both longtitude & latitude
trdata_upd['diag_coord'] = (trdata_upd['Longitude'] + trdata_upd['Latitude'])         # 'diagonal coordinate', works for this coord

# update test data as well
tedata_upd['diag_coord'] = (tedata_upd['Longitude'] + tedata_upd['Latitude'])

features_cols = list(set(trdata_upd.columns)  ^ set([data["target_names"][0]]))
target_cols = data["target_names"][0]

scaler = StandardScaler()
trdata_upd[features_cols] = scaler.fit_transform(trdata_upd[features_cols])
tedata_upd[features_cols] = scaler.transform(tedata_upd[features_cols])


modelEval(trdata_upd, tedata_upd,model_id='dummy')
modelEval(trdata_upd, tedata_upd,model_id='rf')

Scores: [0.9852729  0.94643386 0.98312595 0.98563789 0.98416468], Mean: 0.9769270557701788, std: 0.015272043602982274
Test score: 0.9552074230851957
Scores: [0.48206098 0.4619696  0.45967639 0.47500031 0.47536862], Mean: 0.4708151802139554, std: 0.008567776663917118
Test score: 0.20424811111670002
Scores: [0.9852729  0.94643386 0.98312595 0.98563789 0.98416468], Mean: 0.9769270557701788, std: 0.015272043602982274
Test score: 0.9552074230851957
Scores: [0.48230199 0.46138441 0.45927681 0.47437758 0.4752871 ], Mean: 0.47052557928773836, std: 0.008789648357027025
Test score: 0.20429300755408383
